<a href="https://colab.research.google.com/github/m0rTI88/weather-dashboard-react/blob/main/Advanced_ATM_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json, os
from datetime import datetime, date

# ========== BANK CLASS ==========
class Bank:
    def __init__(self, db_file="bank_data.json"):
        self.db_file = db_file
        self.data = self.load_data()

    def load_data(self):
        if not os.path.exists(self.db_file):
            data = {
                "admins": {
                    "admin": "admin123"
                },
                "users": {},
                "atm_cash": {
                    "2000": 10,
                    "500": 20,
                    "200": 30,
                    "100": 40
                }
            }
            with open(self.db_file, "w") as f:
                json.dump(data, f, indent=4)
            return data
        with open(self.db_file, "r") as f:
            return json.load(f)

    def save_data(self):
        with open(self.db_file, "w") as f:
            json.dump(self.data, f, indent=4)

    # ---- Admin ----
    def validate_admin(self, username, password):
        return self.data["admins"].get(username) == password

    def add_user(self, card, pin, name):
        if card in self.data["users"]:
            return False, "Card already exists"
        self.data["users"][card] = {
            "name": name,
            "pin": str(pin),
            "status": "active",
            "failed_attempts": 0,
            "accounts": {
                "savings": 0,
                "current": 0
            },
            "transactions": [],
            "withdrawn_today": 0,
            "last_withdraw_date": None
        }
        self.save_data()
        return True, "User added"

    def get_user(self, card):
        return self.data["users"].get(card)

    def lock_card(self, card):
        u = self.get_user(card)
        if not u:
            return False
        u["status"] = "locked"
        self.save_data()
        return True

    def unlock_card(self, card):
        u = self.get_user(card)
        if not u:
            return False
        u["status"] = "active"
        u["failed_attempts"] = 0
        self.save_data()
        return True

    # ---- PIN / login ----
    def check_pin(self, card, pin):
        user = self.get_user(card)
        if not user:
            return False, "Card not found"
        if user["status"] == "locked":
            return False, "Card is locked"
        if user["pin"] == str(pin):
            user["failed_attempts"] = 0
            self.save_data()
            return True, "OK"
        user["failed_attempts"] += 1
        if user["failed_attempts"] >= 3:
            user["status"] = "locked"
            self.save_data()
            return False, "Card locked due to too many wrong attempts"
        self.save_data()
        return False, "Wrong PIN"

    # ---- Accounts ----
    def get_balance(self, card, account):
        user = self.get_user(card)
        if not user:
            return None
        return user["accounts"].get(account)

    def deposit(self, card, account, amount):
        user = self.get_user(card)
        if not user or account not in user["accounts"] or amount <= 0:
            return False
        if user["status"] != "active":
            return False
        user["accounts"][account] += amount
        self._add_tx(user, account, "Deposit", amount)
        self.save_data()
        return True

    # ---- ATM cash / withdraw ----
    def calculate_notes(self, amount):
        atm = self.data["atm_cash"]
        notes_needed = {}
        remaining = amount
        for note in [2000, 500, 200, 100]:
            count = min(remaining // note, atm.get(str(note), 0))
            if count > 0:
                notes_needed[note] = count
                remaining -= note * count
        return notes_needed if remaining == 0 else None

    def _reset_withdrawn_if_new_day(self, user):
        today = date.today().isoformat()
        if user["last_withdraw_date"] != today:
            user["last_withdraw_date"] = today
            user["withdrawn_today"] = 0

    def withdraw(self, card, account, amount, daily_limit=5000):
        user = self.get_user(card)
        if not user:
            return False, "Card not found"
        if user["status"] != "active":
            return False, "Card is locked"
        if account not in user["accounts"]:
            return False, "No such account"
        if amount <= 0:
            return False, "Amount must be positive"
        if user["accounts"][account] < amount:
            return False, "Insufficient funds"

        self._reset_withdrawn_if_new_day(user)
        if user["withdrawn_today"] + amount > daily_limit:
            return False, f"Daily limit {daily_limit} exceeded"

        notes = self.calculate_notes(amount)
        if notes is None:
            return False, "ATM cannot dispense this amount with available notes"

        user["accounts"][account] -= amount
        user["withdrawn_today"] += amount
        for note, cnt in notes.items():
            self.data["atm_cash"][str(note)] -= cnt

        self._add_tx(user, account, "Withdrawal", amount, extra={"notes": notes})
        self.save_data()
        return True, notes

    # ---- Transfer ----
    def transfer(self, from_card, to_card, account, amount):
        sender = self.get_user(from_card)
        receiver = self.get_user(to_card)
        if not sender or not receiver:
            return False
        if sender["status"] != "active":
            return False
        if account not in sender["accounts"]:
            return False
        if amount <= 0 or sender["accounts"][account] < amount:
            return False
        sender["accounts"][account] -= amount
        receiver["accounts"]["savings"] += amount
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        sender["transactions"].append({
            "time": now,
            "type": f"Transfer to {to_card}",
            "account": account,
            "amount": amount
        })
        receiver["transactions"].append({
            "time": now,
            "type": f"Received from {from_card}",
            "account": "savings",
            "amount": amount
        })
        self.save_data()
        return True

    def _add_tx(self, user, account, tx_type, amount, extra=None):
        tx = {
            "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "type": tx_type,
            "account": account,
            "amount": amount
        }
        if extra:
            tx.update(extra)
        user["transactions"].append(tx)

    def get_transactions(self, card, limit=None):
        user = self.get_user(card)
        if not user:
            return []
        txs = user["transactions"]
        if limit:
            return txs[-limit:]
        return txs


# ========== ATM CLASS ==========
class ATM:
    def __init__(self, bank):
        self.bank = bank

    def user_login(self):
        card = input("Enter card number: ")
        user = self.bank.get_user(card)
        if not user:
            print("❌ Card not found")
            return None, None
        if user["status"] == "locked":
            print("❌ This card is locked.")
            return None, None

        for _ in range(3):
            pin = input("Enter PIN: ")
            ok, msg = self.bank.check_pin(card, pin)
            if ok:
                print(f"✅ Welcome, {user.get('name','user')}!\n")
                return card, user
            else:
                print("⚠", msg)
                if "locked" in msg.lower():
                    return None, None
        return None, None


# ========== MAIN FUNCTION ==========
def main():
    bank = Bank()
    atm = ATM(bank)

    while True:
        print("""
============================
      ADVANCED ATM SYSTEM
============================
1. User Login
2. Admin Login
3. Exit
""")
        try:
            choice = int(input("Choose: "))
        except:
            print("⚠ Invalid input! Enter a number.")
            continue

        if choice == 1:
            # USER LOGIN
            card, user = atm.user_login()
            if not user:
                continue

            # USER MENU
            while True:
                print("""
========= USER MENU =========
1. Check Balance
2. Deposit Money
3. Withdraw Money
4. Transfer to Another Card
5. View Transactions
6. Change PIN
7. Transfer Between Own Accounts
8. Logout
==============================
""")
                try:
                    option = int(input("Choose option: "))
                except:
                    print("⚠ Enter a valid number")
                    continue

                if option == 1:
                    acct = input("Account (savings/current): ")
                    bal = bank.get_balance(card, acct)
                    if bal is None:
                        print("❌ Unknown account")
                    else:
                        print(f"Balance on {acct}: {bal}")

                elif option == 2:
                    acct = input("Account (savings/current): ")
                    try:
                        amt = int(input("Amount: "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    if bank.deposit(card, acct, amt):
                        print("✅ Deposit successful!")
                    else:
                        print("❌ Deposit failed")

                elif option == 3:
                    acct = input("Account (savings/current): ")
                    try:
                        amt = int(input("Amount: "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    ok, msg = bank.withdraw(card, acct, amt)
                    if ok:
                        print(f"✅ Withdraw Successful! Notes: {msg}")
                    else:
                        print("❌", msg)

                elif option == 4:
                    to_card = input("Receiver card number: ")
                    acct = input("Your account (savings/current): ")
                    try:
                        amt = int(input("Amount to transfer: "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    if bank.transfer(card, to_card, acct, amt):
                        print("✅ Transfer successful!")
                    else:
                        print("❌ Transfer failed")

                elif option == 5:
                    txs = bank.get_transactions(card, limit=10)
                    print("\n--- LAST TRANSACTIONS ---")
                    if not txs:
                        print("No transactions yet.")
                    else:
                        for t in txs:
                            print(f"{t['time']} | {t['type']} | {t['account']} | {t['amount']}")
                    print("-------------------------\n")

                elif option == 6:
                    old_pin = input("Enter current PIN: ")
                    ok, msg = bank.check_pin(card, old_pin)
                    if not ok:
                        print("❌", msg)
                        continue
                    new_pin = input("New PIN: ")
                    confirm = input("Confirm new PIN: ")
                    if new_pin != confirm:
                        print("❌ PINs do not match")
                        continue
                    user["pin"] = str(new_pin)
                    bank.save_data()
                    print("✅ PIN updated")

                elif option == 7:
                    from_acc = input("From (savings/current): ")
                    to_acc = input("To (savings/current): ")
                    try:
                        amt = int(input("Amount: "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    # simple internal transfer
                    u = bank.get_user(card)
                    if (from_acc not in u["accounts"] or
                        to_acc not in u["accounts"] or
                        u["accounts"][from_acc] < amt or amt <= 0):
                        print("❌ Cannot transfer")
                    else:
                        u["accounts"][from_acc] -= amt
                        u["accounts"][to_acc] += amt
                        bank.save_data()
                        print("✅ Transfer between accounts done")

                elif option == 8:
                    break

                else:
                    print("Invalid option!")

        elif choice == 2:
            # ADMIN LOGIN
            username = input("Username: ")
            password = input("Password: ")
            if not bank.validate_admin(username, password):
                print("Wrong credentials")
                continue
            print("Admin logged in!")

            while True:
                print("""
===== ADMIN PANEL =====
1. Add User
2. Refill ATM Cash
3. Check ATM Cash
4. Exit Admin Panel
=======================
""")
                try:
                    admin_option = int(input("Choose: "))
                except:
                    print("Invalid")
                    continue

                if admin_option == 1:
                    card = input("New user card number: ")
                    pin = input("Set PIN: ")
                    name = input("User name: ")
                    ok, msg = bank.add_user(card, pin, name)
                    print("✅" if ok else "❌", msg)

                elif admin_option == 2:
                    print("Current ATM cash:", bank.data["atm_cash"])
                    for note in ["2000", "500", "200", "100"]:
                        try:
                            add = int(input(f"Add how many {note} notes? "))
                        except:
                            add = 0
                        bank.data["atm_cash"][note] += add
                    bank.save_data()
                    print("✅ ATM cash updated")

                elif admin_option == 3:
                    total = 0
                    print("\n--- ATM CASH ---")
                    for note, cnt in bank.data["atm_cash"].items():
                        value = int(note) * cnt
                        total += value
                        print(f"{note}: {cnt} notes (value {value})")
                    print(f"TOTAL: {total}")
                    print("----------------\n")

                elif admin_option == 4:
                    break

                else:
                    print("Invalid admin option!")

        elif choice == 3:
            print("Goodbye!")
            break

        else:
            print("Invalid!")


# ========== START PROGRAM ==========
if __name__ == "__main__":
    main()



      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit

Choose: 2
Username: admin
Password: admin123
Admin logged in!

===== ADMIN PANEL =====
1. Add User
2. Refill ATM Cash
3. Check ATM Cash
4. Exit Admin Panel

Choose: 1
New user card number: 1111
Set PIN: 1234
User name: Test
✅ User added

===== ADMIN PANEL =====
1. Add User
2. Refill ATM Cash
3. Check ATM Cash
4. Exit Admin Panel

Choose: 4

      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit

Choose: 1
Enter card number: 1111
Enter PIN: 1234
✅ Welcome, Test!


========= USER MENU =========
1. Check Balance
2. Deposit Money
3. Withdraw Money
4. Transfer to Another Card
5. View Transactions
6. Change PIN
7. Transfer Between Own Accounts
8. Logout

Choose option: 1
Account (savings/current): savings
Balance on savings: 0

========= USER MENU =========
1. Check Balance
2. Deposit Money
3. Withdraw Money
4. Transfer to Another Card
5. View Transactions
6. Change PIN
7. Transfer Between Own Accounts
8. Logout

Choose